<a href="https://colab.research.google.com/github/LukaLujan/word2vec/blob/main/gthb4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

As I said because RAM "issues" , we will do training and testing in this separated colab notebook

In [ ]:
import numpy as np
import pandas as pd


import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Input, Dense, LSTM, SpatialDropout1D, Dropout, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# I am loading train data that has been preprocessed , embedded , padded and converted into numpy arrays, so it is ready for traning our model.
X_train = np.load("drive/MyDrive/X_train_extra.npy", mmap_mode='r')
y_train = np.load("drive/MyDrive/y_train_extra.npy", mmap_mode='r')

In [ ]:
X_train.shape

(18285, 47, 300)

In [ ]:
y_train.shape

(18285,)

In [ ]:
#We will load test data as well!
X_test = np.load("drive/MyDrive/X_test_extra.npy", mmap_mode='r')
y_test = np.load("drive/MyDrive/y_test_extra.npy", mmap_mode='r')

In [ ]:
X_test.shape, y_test.shape

((4575, 47, 300), (4575,))

In [ ]:
#Spliting our train data, into train and validation set.
X1_train, X1_val, y1_train, y1_val  =train_test_split(X_train, y_train, test_size=0.33, random_state=42)

In [ ]:
print(X1_train.shape, X1_val.shape, y1_train.shape, y1_val.shape)

(12250, 47, 300) (6035, 47, 300) (12250,) (6035,)


In [ ]:
input_shape = (X1_train.shape[1], X1_train.shape[2])

In [ ]:
#Here we are using same - simple model as before. You can play on your own to get better results. Goal was more on preprocessing and converting data into array.

model = Sequential()
# model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
    # here we do not need Embedding layer because we did it on our own) 
model.add(Input(shape=input_shape))
    # https://keras.io/api/layers/core_layers/input/

model.add(LSTM(64))
model.add(Dropout(0.4))
model.add(Dense(32))
model.add(Dropout(0.4))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 64)                93440     
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 95,553
Trainable params: 95,553
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
history=model.fit(X1_train,y1_train, validation_data=(X1_val, y1_val), epochs=10, batch_size=64, callbacks=[EarlyStopping(monitor='val_loss', patience=3,restore_best_weights=True)] )

Epoch 1/10
192/192 [==============================] - 21s 99ms/step - loss: 0.2987 - accuracy: 0.8787 - val_loss: 0.2646 - val_accuracy: 0.8969
Epoch 2/10
192/192 [==============================] - 15s 77ms/step - loss: 0.2689 - accuracy: 0.9024 - val_loss: 0.2624 - val_accuracy: 0.8973
Epoch 3/10
192/192 [==============================] - 15s 79ms/step - loss: 0.3849 - accuracy: 0.8144 - val_loss: 0.3257 - val_accuracy: 0.8832
Epoch 4/10
192/192 [==============================] - 15s 80ms/step - loss: 0.3086 - accuracy: 0.8937 - val_loss: 0.2909 - val_accuracy: 0.8921
Epoch 5/10
192/192 [==============================] - 15s 80ms/step - loss: 0.2690 - accuracy: 0.9013 - val_loss: 0.2719 - val_accuracy: 0.8963


In [ ]:
y_preds = model.predict(X_test)

In [ ]:
for i in range(len(y_preds)):
  if y_preds[i] >=0.5:
    y_preds[i]=1
  else:
    y_preds[i] =0

In [ ]:
y_preds=y_preds.flatten()
pd.Series(y_preds).value_counts()

1.0    2410
0.0    2165
dtype: int64

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
print(accuracy_score(y_test, y_preds))

0.6751912568306011


In [ ]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.67      0.65      0.66      2213
           1       0.68      0.70      0.69      2362

    accuracy                           0.68      4575
   macro avg       0.67      0.67      0.67      4575
weighted avg       0.68      0.68      0.68      4575



As we can see there are somehow better results when we combine both models. Of course if we want to get better results, we would need more data to train, and experiment with much newer and better models. That is for the next time.